In [1]:
from Bio import SeqIO
import pandas as pd
from Bio import Entrez
from Bio import SearchIO
from Bio.Blast import NCBIWWW, NCBIXML
Entrez.email = 'zivse@post.bgu.ac.il' # Enter your email address here
Entrez.api_key = '016d35b4600f9c5d1d5ced586898c3ff3a09' # Enter your API key here

In [2]:
# Fetch the handle for the protein, based on protein ID as a fasta file
ID = "Q96EL2" # Test ID
handle = Entrez.efetch(db="nucleotide", id = 'NC_003522.1', rettype = 'fasta', retmode = 'text')
# Save handle as a fasta file
output = open('test.fasta', 'w')
output.write(handle.read())
output.close()
# Close the handle
handle.close()

# Read the record after it has been saved as a file.
record = SeqIO.read('test.fasta', 'fasta')
record.seq

Seq('GGTAGAGAGGACGAAGGGTAAGTCGTCGGGCTCATGCCCCGAAGAAGCGGGTTC...GGA')

In [3]:
# Run the BLASTp search against a database of all RefSeq proteins
# Currently set to run against the RefSeq protein database and return atleast 500 hits
result_handle = NCBIWWW.qblast(program = "blastp" ,database = "refseq_protein", sequence = record.seq, hitlist_size = 500)

In [4]:
# Save the results of the search as an XML file MAKE SURE YOU SAVE SINCE RUNTIME IS LONG
with open('test.xml', 'w') as f:
    f.write(result_handle.read())
result_handle.close()

In [67]:
# Parse the BLASTp results which were saved into xml file as a BLAST record object
result_handle = open('test.xml')
blast_records = NCBIXML.read(result_handle)
result_handle.close()

In [92]:
# iterate over all hits, print information, generate a report dataframe and a fasta of the protein sequence of all hits
report = {i:[] for i in ['query_id', 'protein_name', 'organism', 'e-value', 'score']} # initialize a dictionary to store the report data
sqeuence_fasta = open('hits.fasta', 'w') # initialize a fasta file to store the protein sequences of all hits
for alignment in blast_records.alignments: # iterate over all hits
    for hsp in alignment.hsps: # iterate over all hsps
        if hsp.expect < 0.05: # if the e-value is less than 0.05
            print('****Alignment****')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print('match:', hsp.match)
            print('sbjct:', hsp.sbjct)
            report['query_id'].append(alignment.title.split('|')[1]) # add the query id to the report dictionary
            report['protein_name'].append(alignment.title.split('|')[2].split(',')[0]) # add the protein name to the report dictionary
            report['organism'].append(alignment.title.split('[')[1].split(']')[0]) # add the organism to the report dictionary
            report['e-value'].append(hsp.expect) # add the e-value to the report dictionary
            report['score'].append(hsp.score) # add the alignment score to the report dictionary
            sqeuence_fasta.write('>' + alignment.title + '\n' + hsp.sbjct + '\n') # add the protein sequence to the fasta file
report_df = pd.DataFrame(report) # convert the report dictionary to a dataframe
report_df.to_csv('test.csv', index=False) # save the report dataframe to a csv file
sqeuence_fasta.close() # close the fasta file handle

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'align_length', 'bits', 'expect', 'frame', 'gaps', 'identities', 'match', 'num_alignments', 'positives', 'query', 'query_end', 'query_start', 'sbjct', 'sbjct_end', 'sbjct_start', 'score', 'strand']
****Alignment****
sequence: ref|NP_114403.1| 28S ribosomal protein S24, mitochondrial precursor [Homo sapiens]
length: 167
e value: 8.0017e-121
match: MAASVCSGLLGPRVLSWSRELPCAWRALHTSPVCAKNRAARVRVSKGDKPVTYEEAHAPHYIAHRKGWLSLHTGNLDGEDHAAERTVEDVFLRKFMWGTFPGCLADQLVLKRRGNQLEICAVVLRQLSPHKYYFLVGYSETLLSYFYKCPVRLHLQTVPSKVVYKYL
sbjct: MAASVCSGLLGPRVLSWSRELPCAWRALHTSPVCAKNRAARVRVSKGDKPVTYEEAHAPHYIAHRKGWLSLHTGNLDGEDHAAERTVEDVFLRKFMWGTFPGCLADQLVLKRRGNQLEICAVVLRQLSPHK